# Лабораторная работа №1
## Задание:
1. Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).
2. Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Лабораторная работа:
### Подключаем библиотеки


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

### Загружаем данные


In [7]:
Dataset = pd.read_csv('survey.csv')
print(Dataset.head())

   Unnamed: 0  Respondent                      MainBranch Hobbyist  \
0           3           4  I am a developer by profession       No   
1           6           7  I am a developer by profession       No   
2          12          13  I am a developer by profession      Yes   
3          21          22  I am a developer by profession      Yes   
4          22          23  I am a developer by profession      Yes   

                                         OpenSourcer  \
0                                              Never   
1                                              Never   
2  Less than once a month but more than once per ...   
3                            Less than once per year   
4                            Less than once per year   

                                          OpenSource  \
0  The quality of OSS and closed source software ...   
1  The quality of OSS and closed source software ...   
2  OSS is, on average, of HIGHER quality than pro...   
3  OSS is, on aver

### Создаём искомый столбец

In [8]:
label_encoder = LabelEncoder()
Dataset['CompTotal Encoded'] = label_encoder.fit_transform(Dataset['CompTotal'])

### Удаляем не числовые столбцы и доплняем числовые

In [10]:
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
columns_to_fill = Dataset.select_dtypes(include=['number']).columns.tolist()
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19490 entries, 0 to 19489
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         19490 non-null  int64  
 1   Respondent         19490 non-null  int64  
 2   CompTotal          19490 non-null  float64
 3   ConvertedComp      19490 non-null  float64
 4   WorkWeekHrs        19490 non-null  float64
 5   CodeRevHrs         19490 non-null  float64
 6   Age                19490 non-null  float64
 7   CompTotal Encoded  19490 non-null  int64  
dtypes: float64(5), int64(3)
memory usage: 1.2 MB


### Ищем лучшие метрики и лучшее количество соседей

In [12]:
X = Dataset.drop(columns=['CompTotal Encoded'])
y = Dataset['CompTotal Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': range(3, 40),
              'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_metric = best_params['metric']
best_k = best_params['n_neighbors']

accuracies = grid_search.cv_results_['mean_test_score']
accuracies = np.array(accuracies).reshape(len(param_grid['n_neighbors']), len(param_grid['metric']))

print(f"Лучшие параметры: Количество соседей = {best_k}, Лучшая метрика = {best_metric}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Лучшие параметры: Количество соседей = 4, Лучшая метрика = manhattan


#### Повторяем обучение на лучших метриках и количестве соседей

In [13]:
knn = KNeighborsClassifier(n_neighbors=best_k, metric=best_metric)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели с 37 соседями и метрикой euclidean: {accuracy}")
report = classification_report(y_test, y_pred, zero_division = 1)
print(report)

Точность модели с 37 соседями и метрикой euclidean: 0.6802206259620318
              precision    recall  f1-score   support

           0       0.09      0.17      0.12        18
           1       0.00      1.00      0.00         0
           2       1.00      0.00      0.00         1
           3       1.00      0.00      0.00         1
           4       1.00      0.00      0.00         1
           5       0.00      1.00      0.00         0
           6       0.00      1.00      0.00         0
           7       0.00      1.00      0.00         0
           8       0.00      1.00      0.00         0
           9       0.00      1.00      0.00         0
          10       0.00      1.00      0.00         0
          12       1.00      0.00      0.00         1
          13       1.00      0.00      0.00         2
          14       0.00      0.00      0.00         3
          15       1.00      0.00      0.00         3
          16       0.00      1.00      0.00         0
          